In [5]:
#Obtention du niveau de diplôme de la population par région en 2022 à partir d'un jeu de données Insee 
# https://catalogue-donnees.insee.fr/fr/catalogue/recherche/DS_RP_DIPLOMES_PRINC

#Il y a plusieurs étapes:
# 1 Le jeu de données de l'INSEE est trop lourd, donc j'ai sélectionné les colonnes nécessaires et sauvegardé un nouveau fichier CSV "diplome_filtered.csv"
# 2 Charger le fichier CSV "diplome_filtered.csv"
# 3. Agréger les données par région pour obtenir le nombre total de diplômés du supérieur
# 4. Calculer le pourcentage de la population ayant un diplôme du supérieur par région


In [6]:
import pandas as pd

diplome=pd.read_csv(r"C:/Users\thoma\OneDrive\Documents\ENSAE\2A\Python-pour-la-data-science\Projet\python_DS_2A\data\data_outdated\diplome_filtered.csv", sep=';')

In [7]:
# Calculer le nombre total de diplômés du supérieur par région (2022)
# S'assurer que la valeur est numérique
diplome['OBS_VALUE'] = pd.to_numeric(diplome['OBS_VALUE'], errors='coerce')

# Codes demandés
codes = ['700_RP', '350T351_RP', '001T100_RP']

# Filtrer et agréger (pivot pour avoir une colonne par code)
diplome_sel = diplome[diplome['EDUC'].isin(codes)].copy()
diplome_region = diplome_sel.pivot_table(index='region_nom', columns='EDUC', values='OBS_VALUE', aggfunc='sum', fill_value=0).reset_index()

# Renommer les colonnes pour être lisibles
diplome_region = diplome_region.rename(columns={
    '700_RP': 'Diplôme de niveau BAC + 5 ou plus',
    '350T351_RP': 'Baccalauréat / Brevet professionnel ou équivalent',
    '001T100_RP': 'Aucun diplôme ou certificat d etudes primaires'
})

# S'assurer que toutes les colonnes existent
for col in ['Diplôme de niveau BAC + 5 ou plus', 'Baccalauréat / Brevet professionnel ou équivalent', 'Aucun diplôme ou certificat d etudes primaires']:
    if col not in diplome_region.columns:
        diplome_region[col] = 0

# Trier par BAC+5 décroissant
diplome_region = diplome_region.sort_values('Diplôme de niveau BAC + 5 ou plus', ascending=False).reset_index(drop=True)


diplome_region.head()

EDUC,region_nom,Aucun diplôme ou certificat d etudes primaires,Baccalauréat / Brevet professionnel ou équivalent,Diplôme de niveau BAC + 5 ou plus
0,Île-de-France,3.056102e+06,2.834181e+06,3.977869e+06
1,Auvergne-Rhône-Alpes,2.202457e+06,2.126194e+06,1.472198e+06
2,Occitanie,1.684524e+06,1.716281e+06,1.035450e+06
3,Provence-Alpes-Côte d'Azur,1.519547e+06,1.460081e+06,9.374938e+05
4,Nouvelle-Aquitaine,1.750780e+06,1.734826e+06,8.806562e+05


In [8]:
#Modifier le dataset pour diviser par le nombre de personnes de plus de 15 ans dans chaque région
# pour obtenir le pourcentage de personnes diplômées du supérieur, la population totale est dans le dataset population_par_région.ipynb

# Charger la population par région (fichier population_par_region_annee.csv)
df_population_region=pd.read_csv(r"C:/Users/thoma/OneDrive/Documents/ENSAE/2A/Python-pour-la-data-science/Projet/python_DS_2A/data/Variables explicatives/population_par_region_annee.csv", sep=',')
# Si le dataset contient plusieurs années, filtrer pour 2022
if 'TIME_PERIOD' in df_population_region.columns:
    df_population_region = df_population_region[df_population_region['TIME_PERIOD'] == 2022].copy()
# Garder les colonnes utiles et forcer en numérique
df_population_region = df_population_region[['region_nom', 'population']].copy()
df_population_region['population'] = pd.to_numeric(df_population_region['population'], errors='coerce')

# Fusionner par région et calculer le taux de diplômés BAC +5 ou plus
diplome_region = diplome_region.merge(df_population_region, on='region_nom', how='left')


# Calculer le taux en protégeant contre la division par zéro
diplome_region['Pourcentage_diplomes_superieur'] = (diplome_region['Diplôme de niveau BAC + 5 ou plus'] / diplome_region['population']) * 100
diplome_region['Pourcentage_diplomes_superieur'] = diplome_region['Pourcentage_diplomes_superieur'].replace([pd.NA, float('inf'), -float('inf')], pd.NA).round(2)

diplome_region['Pourcentage_aucun_diplome_ou_certificat_d_etudes_primaires'] = (diplome_region['Aucun diplôme ou certificat d etudes primaires'] / diplome_region['population']) * 100
diplome_region['Pourcentage_aucun_diplome_ou_certificat_d_etudes_primaires'] = diplome_region['Pourcentage_aucun_diplome_ou_certificat_d_etudes_primaires'].replace([pd.NA, float('inf'), -float('inf')], pd.NA).round(2)

diplome_region['Pourcentage_Baccalauréat / Brevet professionnel ou équivalent'] = (diplome_region['Baccalauréat / Brevet professionnel ou équivalent'] / diplome_region['population']) * 100
diplome_region['Pourcentage_Baccalauréat / Brevet professionnel ou équivalent'] = diplome_region['Pourcentage_Baccalauréat / Brevet professionnel ou équivalent'].replace([pd.NA, float('inf'), -float('inf')], pd.NA).round(2)


diplome_region = diplome_region.sort_values('Pourcentage_diplomes_superieur', ascending=False).reset_index(drop=True)
diplome_region = diplome_region.sort_values('Pourcentage_aucun_diplome_ou_certificat_d_etudes_primaires', ascending=False).reset_index(drop=True)

diplome_region = diplome_region.sort_values('Pourcentage_diplomes_superieur', ascending=False).reset_index(drop=True)
diplome_region = diplome_region.sort_values('Pourcentage_aucun_diplome_ou_certificat_d_etudes_primaires', ascending=False).reset_index(drop=True)

In [9]:
#diplome_region en csv dans le dossier data

diplome_region.to_csv(r"C:/Users/thoma/OneDrive/Documents/ENSAE/2A/Python-pour-la-data-science/Projet/python_DS_2A/data/Variables explicatives/diplome_region.csv", index=False)